In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
import joblib

# Cargar datos
df = pd.read_csv('telecust1000_transformed.csv')

# Preprocesamiento
X = df.drop('custcat', axis=1)
y = df['custcat']

# Normalización
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Codificar etiquetas
le = LabelEncoder()
y_encoded = le.fit_transform(y)

class TelecomDataset(Dataset):
    def __init__(self, X, y):
        # Asegurar que los tensores sean float32
        self.X = torch.FloatTensor(X.astype(np.float32))
        self.y = torch.LongTensor(y.astype(np.int64))
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class OptimizedNN(nn.Module):
    def __init__(self, input_size):
        super(OptimizedNN, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),
            
            nn.Linear(64, 4)
        )
        
    def forward(self, x):
        return self.layers(x)

# División estratificada de datos
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)

# Crear dataloaders
train_dataset = TelecomDataset(X_train.values, y_train)
test_dataset = TelecomDataset(X_test.values, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Configuración
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OptimizedNN(input_size=X.shape[1]).to(device)

# Calcular pesos de clase y convertir a float32
class_counts = np.bincount(y_train)
class_weights = 1.0 / class_counts
class_weights = torch.FloatTensor(class_weights).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.AdamW(
    model.parameters(),
    lr=0.0001,
    weight_decay=0.01,
    amsgrad=True
)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.2,
    patience=3,
    verbose=True
)

# Entrenamiento
n_epochs = 200
best_acc = 0

print("Iniciando entrenamiento...")
for epoch in range(n_epochs):
    # Modo entrenamiento
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
    
    # Modo evaluación
    model.eval()
    correct = 0
    total = 0
    val_preds = []
    val_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_preds.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    accuracy = correct / total
    scheduler.step(accuracy)
    
    if accuracy > best_acc:
        best_acc = accuracy
        best_model_state = model.state_dict().copy()
    
    if (epoch + 1) % 10 == 0:
        print(f'Época [{epoch+1}/{n_epochs}], Accuracy: {accuracy:.4f}')

# Cargar mejor modelo
model.load_state_dict(best_model_state)

# Evaluación final
model.eval()
test_preds = []
test_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        test_preds.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

print("\nReporte de Clasificación:")
print(classification_report(test_labels, test_preds))

# Guardar modelo
model_data = {
    'model_state': model.state_dict(),
    'input_size': X.shape[1],
    'scaler': scaler,
    'label_encoder': le,
    'feature_names': list(X.columns)
}

joblib.dump(model_data, 'optimized_nn_model.joblib')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Iniciando entrenamiento...
Época [10/200], Accuracy: 0.3700
Época [20/200], Accuracy: 0.3650
Época [30/200], Accuracy: 0.3750
Época [40/200], Accuracy: 0.3650
Época [50/200], Accuracy: 0.3750
Época [60/200], Accuracy: 0.3500
Época [70/200], Accuracy: 0.3700
Época [80/200], Accuracy: 0.3600
Época [90/200], Accuracy: 0.3650
Época [100/200], Accuracy: 0.3700
Época [110/200], Accuracy: 0.3750
Época [120/200], Accuracy: 0.3750
Época [130/200], Accuracy: 0.3950
Época [140/200], Accuracy: 0.3700
Época [150/200], Accuracy: 0.3600
Época [160/200], Accuracy: 0.3800
Época [170/200], Accuracy: 0.3450
Época [180/200], Accuracy: 0.3750
Época [190/200], Accuracy: 0.3550
Época [200/200], Accuracy: 0.3850

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.37      0.36      0.36        53
           1       0.33      0.34      0.34        44
           2       0.49      0.41      0.45        56
           3       0.36      0.43      0.39        47

   

['optimized_nn_model.joblib']